In [1]:
# Import libraries
import pandas as pd
import numpy as np

from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import train_test_split

from utilities import data_basic_utility as databasic

## File Details

Basic run with SVD without cross validation. This can be the basis for parameter tuning and other stuff later


# Assignment 3 -- Recommendation Systems

* The final challenge is much like Assignment 2 -- but scoped appropriately for the time and your current abilities.
* It is ratings prediction, just like the movielens recommendations we have seen and many other similar problems.
* The features created are based on Beer Reviews from experts on a website.
* Each beer has been scored between 0 and 5 (on a real scale, so 2.75 or 3.5 is OK).
* The official measure is Mean Average Error (MAE) which is pretty intuitive to work with. Everything supports is and it is easy to interpret.
* A set of features have been created based on the reviewer, the written review, and information about the Beer being reviewed.
* Not all features have to be used, and you can easily create new features using the data if you like.
* The features included are:

![title](Images/A3Features.png)

* Sizes of the files are:
|Size | File|
|---|---|
| 1.9G | features.tsv |
| 88B  | header-features.tsv|
| 48B  | header.tsv |
| 15M  | test.tsv |
| 50M  | train.tsv |
| 16M |  val.tsv |


In [2]:
filePrefix = "A3_073_surprise_svdpp"
baseDataDir = "C:/Development/Data/COSC2670/Assignment3/A3data/"
subrunDir = "subruns/"
writeSubRunFile = True
seed = databasic.get_random_seed()

In [3]:
# RowID  BeerID  ReviewerID  BeerName  BeerType  Label
#df_train = pd.read_csv(baseDataDir + 'train_wk12.tsv',sep='\t',
df_train = pd.read_csv(baseDataDir + 'train_200k.tsv',sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_train.head(10)

#df_vali = pd.read_csv(baseDataDir + 'val_wk12.tsv',sep='\t',
df_vali = pd.read_csv(baseDataDir + 'vali_200k.tsv',sep='\t',
                         names=['RowID','BeerID','ReviewerID',
                                  'BeerName','BeerType','rating'])
df_vali.head(10)


,RowID,BeerID,ReviewerID,BeerName,BeerType,rating
0,22,12300,2634,Rauch �r Bock,Rauchbier,4.0
1,27,12300,5634,Rauch �r Bock,Rauchbier,4.5
2,28,12300,3544,Rauch �r Bock,Rauchbier,4.5
3,40,12300,6521,Rauch �r Bock,Rauchbier,4.0
4,43,12300,10177,Rauch �r Bock,Rauchbier,4.5
5,48,12300,2907,Rauch �r Bock,Rauchbier,3.5
6,49,12300,1532,Rauch �r Bock,Rauchbier,4.0
7,50,12300,3452,Rauch �r Bock,Rauchbier,3.5
8,59,12300,6861,Rauch �r Bock,Rauchbier,4.0
9,64,6699,6401,Caldera Pale Ale,American Pale Ale (APA),4.5


Column List: 
RowID BrewerID ABV DayofWeek Month DayofMonth Year TimeOfDay Gender Birthday Text Lemmatized POS_Tag

In [4]:
# df_features = pd.read_csv(baseDataDir + 'features-top500.tsv',sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
df_features = pd.read_csv(baseDataDir + 'features_200k.tsv',sep='\t', names=['RowID','BrewerID','ABV','DayofWeek','Month',
                                                                 'DayofMonth','Year','TimeOfDay','Gender',
                                                                 'Birthday','Text','Lemmatized','POS_Tag'])

df_features.head(10)

,RowID,BrewerID,ABV,DayofWeek,Month,DayofMonth,Year,TimeOfDay,Gender,Birthday,Text,Lemmatized,POS_Tag
0,18,1075,7.4,Mon,Jan,2,2012,15:20:04,Male,unknown,Pours a murky light brown with a 1 inch fizzy ...,pour a murky light brown with a 1 inch fizzy t...,VBZ DT JJ NN JJ IN DT CD NN JJ NN NN WDT VBZ I...
1,19,1075,7.4,Sun,Jan,1,2012,06:46:52,Male,unknown,Faint sudsy head with some with some dissipati...,faint sudsy head with some with some dissipate...,NN JJ NN IN DT IN DT VBG JJ NN . JJ JJ NN . DT...
2,20,1075,7.4,Tue,Nov,29,2011,05:51:44,Male,unknown,A new arrival to the West TN area ... Pours qu...,a new arrival to the West TN area ... pour qui...,"DT JJ NN IN DT NNP NNP NN , VBZ PDT DT NN JJR ..."
3,21,1075,7.4,Sat,Nov,5,2011,22:59:57,Male,unknown,Sampled 10/30/11 - Transferring the notes . A ...,sample 10/30/11 - transfer the note . a ruby p...,VBN CD HYPH VBG DT NNS . DT NN VBP IN DT NN NN...
4,22,1075,7.4,Tue,Nov,1,2011,20:40:21,Male,"Oct 14, 1983",This is my first rauchbier . Pours a burnt amb...,this be my first rauchbier . pour a burn amber...,DT VBZ PRP$ JJ NN . VBZ DT VBN NN . JJ NN . NN...
5,23,1075,7.4,Sat,Oct,29,2011,11:18:34,unknown,unknown,A,a,DT
6,24,1075,7.4,Mon,Oct,17,2011,09:04:02,unknown,unknown,"Pours a mahogany color , rich , with a tan hea...","pour a mahogany color , rich , with a tan head...","VBZ DT NN NN , JJ , IN DT JJ NN . DT NN , VBD ..."
7,25,1075,7.4,Tue,Oct,11,2011,10:41:41,unknown,unknown,Pours light caramel brown with reddish highlig...,pour light caramel brown with reddish highligh...,NNS JJ NN JJ IN JJ NNS . DT JJ JJ NN VBZ RB VB...
8,26,1075,7.4,Tue,Oct,11,2011,10:31:22,Male,"Nov 24, 1974",Poured a slightly cloudy deep amber/red color ...,pour a slightly cloudy deep amber/red color wi...,VBD DT RB JJ JJ VBN NN IN DT JJ NN NN IN NN . ...
9,27,1075,7.4,Mon,Sep,19,2011,14:18:29,Male,"Oct 10, 1988",Big thanks to N2168 for knocking this off my w...,big thanks to n2168 for knock this off my want...,JJ NNS IN NN IN VBG DT RP PRP$ NNS . VBN IN DT...


In [5]:
idCols = ['RowID','BeerID','ReviewerID']

# Setup the data to be just the Reviewer and the Beer(Item) and the Rating Label we want to learn.
dfTrainFeatures = df_train.drop(['RowID','BeerName','BeerType'],axis=1)
dfValiIds = df_vali[idCols]
dfValiFeatures = df_vali.drop(['RowID','BeerName','BeerType'],axis=1)

dfTrainFeatures.head()

,BeerID,ReviewerID,rating
0,12300,10635,4.0
1,12300,6547,4.5
2,12300,9789,4.5
3,12300,7372,5.0
4,12300,1302,4.5


In [6]:

reader = Reader(rating_scale=(0, 5))
dsetTrainFeatures = Dataset.load_from_df(dfTrainFeatures[['BeerID','ReviewerID',
                                    'rating']],reader)

dsetValiFeatures = Dataset.load_from_df(dfValiFeatures[['BeerID','ReviewerID',
                                     'rating']],reader)
trainsetTrainFeatures = dsetTrainFeatures.build_full_trainset()

print(type(dsetTrainFeatures))
print(type(trainsetTrainFeatures))
trainsetTrainFeatures

<class 'surprise.dataset.DatasetAutoFolds'>
<class 'surprise.trainset.Trainset'>


In [7]:

NA,valset = train_test_split(dsetValiFeatures, test_size=1.0)

# SVD standard best params, quick and full
# best_params = {'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.4}
# best_params (full) = {'n_factors': 30, 'n_epochs': 20, 'biased': True (default), 'lr_all': 0.005, 'reg_all': 0.1}
# best_params (from test 3) = {'n_factors': 10, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}

# algorithm = SVDpp()
algorithm = SVDpp(n_factors = 10, n_epochs=20, lr_all=0.005, reg_all=0.2)

model = algorithm.fit(trainsetTrainFeatures)
predictions = algorithm.test(valset)


# Score our predictions with MAE
# It is around 0.77, which means the a random guess based on the distribution of the data
# is on average within 0.77 (plus or minus) the true label.
# Not bad! You can beat it though, I'm sure :).
# Smaller MAE is the better. Good luck!
mae = accuracy.mae(predictions,verbose=True)

print("Average MAE: " + str(mae))

# Takes about 8 minutes

MAE:  0.4284
Average MAE: 0.4283718397372383


In [8]:
print(type(valset))
print(valset[0:10])


<class 'list'>
[(8815, 3338, 2.0), (11816, 547, 4.0), (2822, 8921, 4.0), (12620, 1208, 3.5), (7714, 1722, 3.0), (11598, 7278, 4.0), (1644, 8624, 4.0), (6304, 4706, 4.5), (5269, 813, 3.5), (11117, 3057, 4.0)]


In [9]:
print(type(predictions))
print(str(len(predictions)))
print(predictions[0:10])
print(predictions[0])

<class 'list'>
39509
[Prediction(uid=8815, iid=3338, r_ui=2.0, est=3.0450381279480574, details={'was_impossible': False}), Prediction(uid=11816, iid=547, r_ui=4.0, est=4.037409103156431, details={'was_impossible': False}), Prediction(uid=2822, iid=8921, r_ui=4.0, est=3.62057464370558, details={'was_impossible': False}), Prediction(uid=12620, iid=1208, r_ui=3.5, est=3.8765237706061457, details={'was_impossible': False}), Prediction(uid=7714, iid=1722, r_ui=3.0, est=3.960325739981609, details={'was_impossible': False}), Prediction(uid=11598, iid=7278, r_ui=4.0, est=3.743523987637924, details={'was_impossible': False}), Prediction(uid=1644, iid=8624, r_ui=4.0, est=4.1197345049103955, details={'was_impossible': False}), Prediction(uid=6304, iid=4706, r_ui=4.5, est=4.019470743462229, details={'was_impossible': False}), Prediction(uid=5269, iid=813, r_ui=3.5, est=3.9499719356354452, details={'was_impossible': False}), Prediction(uid=11117, iid=3057, r_ui=4.0, est=3.367852714565678, details={

In [10]:
# Convert the Predictions to a dataframe so we can lookup predictions easy
lstUIds = list(map(lambda x: x.uid, predictions))
lstIIds = list(map(lambda x: x.iid, predictions))
lstTrueRatings = list(map(lambda x: x.r_ui, predictions))
lstRatingEst = list(map(lambda x: x.est, predictions))


# uid == BeerId, iid == ReviewerId, r_ui == Original Ration, est = Predicted rating
dfPredictions = pd.DataFrame({ "uid": lstUIds,"iid": lstIIds, "r_ui": lstTrueRatings, "Predict": lstRatingEst })

dfPredictions.head()
# dfPredictions[dfPredictions.uid == 3519]

,uid,iid,r_ui,Predict
0,8815,3338,2.0,3.045038
1,11816,547,4.0,4.037409
2,2822,8921,4.0,3.620575
3,12620,1208,3.5,3.876524
4,7714,1722,3.0,3.960326


In [11]:
print(dfValiIds.shape)
print(dfPredictions.shape)

(39509, 3)
(39509, 4)


In [12]:
# join the predictions to the ids, sort by rowid and write to file
dfPredictions = pd.merge(dfValiIds, dfPredictions, how="inner", left_on=["BeerID", "ReviewerID"], right_on=["uid", "iid"])
dfPredictions.head()


,RowID,BeerID,ReviewerID,uid,iid,r_ui,Predict
0,22,12300,2634,12300,2634,4.0,4.307723
1,27,12300,5634,12300,5634,4.5,4.200102
2,28,12300,3544,12300,3544,4.5,4.335563
3,40,12300,6521,12300,6521,4.0,4.217186
4,43,12300,10177,12300,10177,4.5,4.162843


This basic normal run doesn't give good MAE, so not worth writing out and considering

In [13]:
if writeSubRunFile:
  dfPredictions.sort_values("RowID")[["RowID", "BeerID", "ReviewerID", "Predict"]].to_csv(subrunDir + filePrefix + "_subrun.csv", index=False)

print("Average MAE: " + str(mae))
print("analyse_maes.append(" + str(mae) + ")")
print(dfPredictions.shape)
dfPredictions.sort_values("RowID").head(8)


Average MAE: 0.4283718397372383
analyse_maes.append(0.4283718397372383)
(39703, 7)


,RowID,BeerID,ReviewerID,uid,iid,r_ui,Predict
0,22,12300,2634,12300,2634,4.0,4.307723
1,27,12300,5634,12300,5634,4.5,4.200102
2,28,12300,3544,12300,3544,4.5,4.335563
3,40,12300,6521,12300,6521,4.0,4.217186
4,43,12300,10177,12300,10177,4.5,4.162843
5,48,12300,2907,12300,2907,3.5,3.940503
6,49,12300,1532,12300,1532,4.0,4.225543
7,50,12300,3452,12300,3452,3.5,4.080092


In [14]:

print("Run - " + filePrefix)
# Log of Results
analyse_maes = []

print("Average MAE over all tests: " + str(np.mean(analyse_maes)))

# Make sure it's predicting floats
# dfPredicted["Predict"].drop_duplicates()

Run - A3_073_surprise_svdpp
Average MAE over all tests: nan


C:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Average MAE with simple SVD: 0.43410560647570734

MAE with first run of best params:  0.4311946847059103

MAE with simple SVDpp: 0.429803678058171

MAE with SVDpp run of best params (from SVD tune full params) 0.4281931590617176